In [36]:
# coding: utf8

# Chercher des verbes similaires à clore dans Lexique3

In [1]:
import codecs,math,pickle,re,math,pyperclip
import pandas as pd
import numpy as np
import itertools as it
import matplotlib as mplt
import matplotlib.pyplot as plt

In [2]:
pd.__version__

u'0.23.1'

In [3]:
%matplotlib inline

In [4]:
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.set_palette("hls")
sns.set_color_codes("muted")
#sns.palplot(sns.color_palette("hls", 8))
#sns.palplot(sns.color_palette("husl", 8))

# Préparations
- répertoires
 - HDR
 - Lexique3
 - Verbes3

In [5]:
repHDR="/Users/gilles/ownCloud/Recherche/Boye/HDR/Memoire/figs/"
repDATA="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Longitudinales/"
nomRepertoire="/Volumes/gilles/Transfert/Copies-iMac-GB/Python/phonemisation/"
nomRepertoire="/Users/gilles/Dropbox/Travail/Recherche/BoKu/LingInfo/Lexique3/Lexique381/"
nomLexiqueOrg=nomRepertoire+"Lexique380-UTF8.txt"
nomLexiqueOrg=nomRepertoire+"Lexique381.txt"
Lexique3=pd.read_csv(nomLexiqueOrg,sep="\t",encoding="utf8")
nomVerbes3="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Samples/MGC-171229-Verbes3.pkl"
nomVerbes3Csv="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Samples/MGC-171229-Verbes3.csv"

# Espace thématique des verbes du français

In [6]:
etCells=[
    ["pi1P","pi2P"]+["ii"+p+n for p in "123" for n in "SP"],
    ["pi3P"],
    ["pi"+p+"S" for p in "123"],
    ["pP"],
    ["pI2S"],
    ["pI1P","pI2P"],
    ["ps"+p+"S" for p in "123"]+["ps3P"],
    ["ps1P","ps2P"],
    ["inf"],
    ["fi"+p+n for p in "123" for n in "SP"]+["pc"+p+n for p in "123" for n in "SP"],
    ["ai"+p+n for p in "123" for n in "SP"]+["is"+p+n for p in "123" for n in "SP"],
    ["pp"+g+n for g in "MF" for n in "SP"],
        ]
for nE,e in enumerate(etCells):
    print nE+1,", ".join(e)

1 pi1P, pi2P, ii1S, ii1P, ii2S, ii2P, ii3S, ii3P
2 pi3P
3 pi1S, pi2S, pi3S
4 pP
5 pI2S
6 pI1P, pI2P
7 ps1S, ps2S, ps3S, ps3P
8 ps1P, ps2P
9 inf
10 fi1S, fi1P, fi2S, fi2P, fi3S, fi3P, pc1S, pc1P, pc2S, pc2P, pc3S, pc3P
11 ai1S, ai1P, ai2S, ai2P, ai3S, ai3P, is1S, is1P, is2S, is2P, is3S, is3P
12 ppMS, ppMP, ppFS, ppFP


# Problèmes de codage unicode dans les lexiques

In [7]:
Lexique3[Lexique3["3_lemme"].str.encode("utf8")==u"paître".encode("utf8")]

,1_ortho,2_phon,3_lemme,4_cgram,5_genre,6_nombre,7_freqlemfilms2,8_freqlemlivres,9_freqfilms2,10_freqlivres,...,26_orthrenv,27_phonrenv,28_orthosyll,29_cgramortho,30_deflem,31_defobs,32_old20,33_pld20,34_morphoder,35_nbmorph
91384,pais,pE,paître,VER,NaN,NaN,2.29,4.46,0.39,0.07,...,siap,Ep,pais,VER,88.0,18.0,1.00,1.00,paître,1
91388,paissaient,pEsE,paître,VER,NaN,NaN,2.29,4.46,0.01,1.49,...,tneiassiap,EsEp,pais-saient,VER,88.0,18.0,1.65,1.00,paître,1
91389,paissait,pEsE,paître,VER,NaN,NaN,2.29,4.46,0.14,0.20,...,tiassiap,EsEp,pais-sait,VER,88.0,18.0,1.60,1.00,paître,1
91390,paissant,pEs@,paître,VER,NaN,NaN,2.29,4.46,0.03,0.34,...,tnassiap,@sEp,pais-sant,VER,88.0,18.0,1.50,1.00,paître,1
91391,paissent,pEs,paître,VER,NaN,NaN,2.29,4.46,0.42,0.61,...,tnessiap,sEp,pais-sent,VER,88.0,18.0,1.50,1.00,paître,1
93652,paît,pE,paître,VER,NaN,NaN,2.29,4.46,0.20,0.00,...,tîap,Ep,paît,VER,88.0,18.0,1.70,1.00,paître,1
93653,paître,pEtR,paître,VER,NaN,NaN,2.29,4.46,0.97,1.76,...,ertîap,RtEp,paî-tre,VER,88.0,18.0,1.85,1.10,paître,1
93654,paîtront,pEtR§,paître,VER,NaN,NaN,2.29,4.46,0.12,0.00,...,tnortîap,§RtEp,paî-tront,VER,88.0,18.0,2.50,1.35,paître,1


Avec le changement de version de pandas, les anciens pickle ne sont plus compatibles avec la nouvelle version 0.23.1, pd.read_pickle permet de lire un fichier pkl contenant un DataFrame datant d'une version précédente de pandas mais pas un dictionnaire de DataFrame...

In [8]:
verbes3=pd.read_pickle(nomVerbes3)
verbes3

,ortho,phono,ext,cs,ms,vs,lexeme,freq,prob,case
40,a,a,NaN,V,3S,pi,avoir,7.167735e+11,2.076214e-02,pi3S
45,abaissa,abEsa,NaN,V,3S,ai,abaisser,2.640001e+08,7.647056e-06,ai3S
46,abaissai,abEsE,NaN,V,1S,ai,abaisser,1.700001e+07,4.924243e-07,ai1S
47,abaissaient,abEsE,"t""",V,3P,ii,abaisser,4.100005e+07,1.187612e-06,ii3P
48,abaissais,abEsE,"z""",V,1S,ii,abaisser,4.933548e+01,1.429057e-12,ii1S
49,abaissais,abEsE,"z""",V,2S,ii,abaisser,4.954692e-01,1.435182e-14,ii2S
50,abaissait,abEsE,"t""",V,3S,ii,abaisser,2.520003e+08,7.299467e-06,ii3S
52,abaissant,abEsâ,"t""",V,,pP,abaisser,2.510001e+08,7.270495e-06,pP
56,abaissas,abEsa,"z""",V,2S,ai,abaisser,2.565900e-01,7.432415e-15,ai2S
57,abaissasse,abEsas,@,V,1S,is,abaisser,3.093462e-01,8.960556e-15,is1S


In [9]:
verbes3.loc[:,"lexeme"]=verbes3.loc[:,"lexeme"].str.encode("utf8")
#verbes3.loc[:,"lexeme"]=verbes3.loc[:,"lexeme"].str.decode("utf8")

Il y a un problème avec l'encodage des noms de lexèmes.

- str.encode("utf8") permet de recoder les noms pour pouvoir les sélectionner
- dans les tests, utiliser "paître" sans la notation unicode

In [10]:
casesVerbes=verbes3.case.unique().tolist()

In [11]:
verbesLex3=verbes3[(verbes3["freq"]>10000)]
verbesLex3[verbesLex3["lexeme"]==u"rêver".encode("utf8")]

,ortho,phono,ext,cs,ms,vs,lexeme,freq,prob,case
359382,rêva,rEva,NaN,V,3S,ai,rêver,3.070009e+08,8.892622e-06,ai3S
359383,rêvai,rEvE,NaN,V,1S,ai,rêver,8.500011e+07,2.462122e-06,ai1S
359384,rêvaient,rEvE,"t""",V,3P,ii,rêver,5.050004e+08,1.462790e-05,ii3P
359385,rêvais,rEvE,"z""",V,1S,ii,rêver,2.191201e+09,6.347055e-05,ii1S
359386,rêvais,rEvE,"z""",V,2S,ii,rêver,3.679995e+07,1.065952e-06,ii2S
359387,rêvait,rEvE,"t""",V,3S,ii,rêver,2.224002e+09,6.442069e-05,ii3S
359388,rêvant,rEvâ,"t""",V,,pP,rêver,7.540005e+08,2.184046e-05,pP
359448,rêve,rEv,@,V,1S,pi,rêver,1.460818e+09,4.231421e-05,pi1S
359450,rêve,rEv,@,V,2S,pI,rêver,6.658537e+07,1.928719e-06,pI2S
359451,rêve,rEv,@,V,3S,pi,rêver,4.066605e+09,1.177937e-04,pi3S


# Transcriptions Sampa => IPA

In [12]:
cSAMPA=u"SZNJêôârEHO692"
cIPA=[u"ʃ",u"ʒ",u"ŋ",u"ɲ",u"ɛ̃",u"ɔ̃",u"ɑ̃",u"ʁ",u"ɛ",u"ɥ",u"ɔ",u"ə",u"œ",u"ø"]

In [13]:
import unicodedata
def listerUnicode(chaine):
    result=[]
    comb=False
    prec=u""
    for char in chaine:
        print char,
        if unicodedata.combining(char):
            result.append(prec+char)
            prec=""
        else:
            result.append(prec)
            prec=char
    result.append(prec)
    return [r for r in result if r!=""]

ipaIn = listerUnicode(cSAMPA)
ipaOut= cIPA
toipa = dict(zip(ipaIn, ipaOut))

def coderIPA(chaine,table=toipa):
    result=chaine
    for k in table:
        result=result.replace(k,table[k])
    return result

S Z N J ê ô â r E H O 6 9 2


In [14]:
coderIPA(u"b2v6ri")

u'b\xf8v\u0259\u0281i'

# Coloration des tableaux d'espaces thématiques

In [39]:
etColours=["orange",
           "brown!50",
           "brown",
           "blue!10",
           "yellow!50",
           "blue!25",
           "teal!50",
           "blue!50",
           "cyan!50",
           "lime",
           "pink",
           "magenta!50",
          ]
etCells=[
    ["pi1P","pi2P"]+["ii"+p+n for p in "123" for n in "SP"],
    ["pi3P"],
    ["pi"+p+"S" for p in "123"],
    ["pP"],
    ["pI2S"],
    ["pI1P","pI2P"],
    ["ps"+p+"S" for p in "123"]+["ps3P"],
    ["ps1P","ps2P"],
    ["inf"],
    ["fi"+p+n for p in "123" for n in "SP"]+["pc"+p+n for p in "123" for n in "SP"],
    ["ai"+p+n for p in "123" for n in "SP"]+["is"+p+n for p in "123" for n in "SP"],
    ["pp"+g+n for g in "MF" for n in "SP"],
        ]
tabTemps={
    "pi":u"ind. prs",
    "ii":u"ind. ipf",
    "ai":u"ind. ps",
    "fi":u"ind. fut",
    "ps":u"subj. prs",
    "is":u"subj. ipf",
    "pc":u"cond. prs",
    "pI":u"imper. prs",
    "inf":u"non-fini"
    }
dictEtColours={}
for nET,ET in enumerate(etCells):
    for c in ET:
        dictEtColours[c]=etColours[nET]
dictEtColours

{'ai1P': 'pink',
 'ai1S': 'pink',
 'ai2P': 'pink',
 'ai2S': 'pink',
 'ai3P': 'pink',
 'ai3S': 'pink',
 'fi1P': 'lime',
 'fi1S': 'lime',
 'fi2P': 'lime',
 'fi2S': 'lime',
 'fi3P': 'lime',
 'fi3S': 'lime',
 'ii1P': 'orange',
 'ii1S': 'orange',
 'ii2P': 'orange',
 'ii2S': 'orange',
 'ii3P': 'orange',
 'ii3S': 'orange',
 'inf': 'cyan!50',
 'is1P': 'pink',
 'is1S': 'pink',
 'is2P': 'pink',
 'is2S': 'pink',
 'is3P': 'pink',
 'is3S': 'pink',
 'pI1P': 'blue!25',
 'pI2P': 'blue!25',
 'pI2S': 'yellow!50',
 'pP': 'blue!10',
 'pc1P': 'lime',
 'pc1S': 'lime',
 'pc2P': 'lime',
 'pc2S': 'lime',
 'pc3P': 'lime',
 'pc3S': 'lime',
 'pi1P': 'orange',
 'pi1S': 'brown',
 'pi2P': 'orange',
 'pi2S': 'brown',
 'pi3P': 'brown!50',
 'pi3S': 'brown',
 'ppFP': 'magenta!50',
 'ppFS': 'magenta!50',
 'ppMP': 'magenta!50',
 'ppMS': 'magenta!50',
 'ps1P': 'blue!50',
 'ps1S': 'teal!50',
 'ps2P': 'blue!50',
 'ps2S': 'teal!50',
 'ps3P': 'teal!50',
 'ps3S': 'teal!50'}

# Fabrication des tabulars de conjugaison

In [16]:
def makeTabularParadigmeDF(lexeme,lDF,dictColours,title="",coulLim=False, cat="V",dictMorphomes={}):
    row=lDF[lDF["lexeme"]==lexeme.encode("utf8")]
    return makeTabularParadigme(row,dictColours,title="",coulLim=False, cat="V",dictMorphomes={},lexeme=lexeme)

def makeTabularParadigme(row,dictColours,title="",coulLim=False, cat="V",dictMorphomes={},lexeme=""):
    tabular=[]
    def makeValue(case):
#        if len(row[case])>0 and len(row[case].values[0])>0:
        if case in row and all(row[case].notnull()):
            result=coderIPA(row[case].values[0])
        elif dictMorphomes!={} and case in dictMorphomes:
            altCase=dictMorphomes[case][0]
            if len(row[altCase])>0 and len(row[altCase].values[0])>0:
                result=coderIPA(row[altCase].values[0])
            else:
                result="?"
        else:
            result="?"
        return result
    
    def makeLine6(tenseCode):
        line=[tabTemps[tenseCode]]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            case=tenseCode+person
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
        return r" & ".join(line)+r"\\"

    def makeLine3(tenseCode):
        line=[tabTemps[tenseCode]]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            if person in ["2S","1P","2P"]:
                case=tenseCode+person
                if case in dictColours:
                    line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
                else:
                    line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
#                line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
            else:
                line.append(r"---")
        return r" & ".join(line)+r"\\"
    
    def makeLineNF():
        line=["non-fini"]
        for case in ["inf","pP","ppMS","ppMP","ppFS","ppFP"]:
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
#            line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
        return r" & ".join(line)+r"\\"

    def makeLineMF(nombre):
        line=[]
        for genre in "mf":
            case=genre+nombre
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
        return r" & ".join(line)+r"\\"
    
    def makeLineCoulLim():
        line=[]
        for numLimite,limite in enumerate(listLimites):
            line.append(r"\cellcolor{%s}%s"%(listLimCoul[numLimite],"$<$"+str(limite)))
        return r"\hline\hline "+r" & ".join(line)+r"\\"
        
    if cat=="V":
        top=[
            r"\begin{center}",
            r"\begin{tabular}{ccccccc}",
            r"\toprule",
            " & ".join([ur"\textsc{%s}"%lexeme]+[p+n for n in ["sg","pl"] for p in "123" ])+r"\\",
            r"\midrule"
            ]
        bottom=[
            r"\bottomrule",
            r"\end{tabular}\\",
            title,
            r"\end{center}",
            r"\bigskip",
            r""
            ]
        tabular.append("\n".join(top))
        for tenseCode in ["pi","ii","fi","pc", "ps","ai", "is"]:
            tabular.append(makeLine6(tenseCode))
        tabular.append(makeLine3("pI"))
        tabular.append(u"\midrule\n")
        tabular.append(ur"& inf. & part. prés. & \multicolumn{4}{c}{part. passé}\\")
        tabular.append(makeLineNF())
    elif cat=="A":
        top=[
            r"\begin{center}",
            r"\begin{tabular}{cc}",
            r"\toprule"
            ]
        bottom=[
            r"\bottomrule",
            r"\end{tabular}\\",
            title,
            r"\end{center}",
#            r"\bigskip",
            r""
            ]
        tabular.append("\n".join(top))
        for number in "sp":
            tabular.append(makeLineMF(number))
    if coulLim:
        tabular.append(makeLineCoulLim())
    tabular.append("\n".join(bottom))
    return "\n".join(tabular)

# Paradigmes du Gold Lexique3

In [17]:
dfGold3=verbes3.pivot_table(values='phono', 
                                 index=['lexeme'], 
                                 columns=['case'], 
                                 aggfunc=lambda x: ",".join(x)
                                ).reset_index().reindex()
dfGold3

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abaisser,abEsam,abEsE,abEsat,abEsa,abEsEr,abEsa,abEs6rô,abEs6rE,abEs6re,...,abEse,abEse,abEse,abEse,abEsjô,abEs,abEsje,abEs,abEs,abEs
1,abandonner,abâdOnam,abâdOnE,abâdOnat,abâdOna,abâdOnEr,abâdOna,abâdOn6rô,abâdOn6rE,abâdOn6re,...,abâdOne,abâdOne,abâdOne,abâdOne,abâdOnjô,abâdOn,abâdOnje,abâdOn,abâdOn,abâdOn
2,abasourdir,abazurdim,abazurdi,abazurdit,abazurdi,abazurdir,abazurdi,abazurdirô,abazurdirE,abazurdire,...,abazurdi,abazurdi,abazurdi,abazurdi,abazurdisjô,abazurdis,abazurdisje,abazurdis,abazurdis,abazurdis
3,abattre,abatim,abati,abatit,abati,abatir,abati,abatrô,abatrE,abatre,...,abaty,abaty,abaty,abaty,abatjô,abat,abatje,abat,abat,abat
4,abcéder,absEdam,absEdE,absEdat,absEda,absEdEr,absEda,absEd6rô,absEd6rE,absEd6re,...,NaN,NaN,NaN,absEde,absEdjô,absEd,absEdje,absEd,absEd,absEd
5,abdiquer,abdikam,abdikE,abdikat,abdika,abdikEr,abdika,abdik6rô,abdik6rE,abdik6re,...,abdike,abdike,abdike,abdike,abdikjô,abdik,abdikje,abdik,abdik,abdik
6,abecquer,abEkam,abEkE,abEkat,abEka,abEkEr,abEka,abEk6rô,abEk6rE,abEk6re,...,abEke,abEke,abEke,abEke,abEkjô,abEk,abEkje,abEk,abEk,abEk
7,aberrer,abEram,abErE,abErat,abEra,abErEr,abEra,abEr6rô,abEr6rE,abEr6re,...,NaN,NaN,NaN,abEre,abErjô,abEr,abErje,abEr,abEr,abEr
8,abhorrer,abOram,abOrE,abOrat,abOra,abOrEr,abOra,abOr6rô,abOr6rE,abOr6re,...,abOre,abOre,abOre,abOre,abOrjô,abOr,abOrje,abOr,abOr,abOr
9,abjurer,abZyram,abZyrE,abZyrat,abZyra,abZyrEr,abZyra,abZyr6rô,abZyr6rE,abZyr6re,...,abZyre,abZyre,abZyre,abZyre,abZyrjô,abZyr,abZyrje,abZyr,abZyr,abZyr


# Paradigmes de Lexique3 (Verbes3)

In [18]:
dfParadigmes3=verbesLex3.pivot_table(values='phono', 
                                 index=['lexeme'], 
                                 columns=['case'], 
                                 aggfunc=lambda x: ",".join(x)
                                ).reset_index().reindex()
dfParadigmes3

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abaisser,NaN,abEsE,NaN,NaN,abEsEr,abEsa,NaN,abEs6rE,NaN,...,abEse,abEse,abEse,abEse,NaN,NaN,NaN,NaN,NaN,NaN
1,abandonner,abâdOnam,abâdOnE,NaN,abâdOna,abâdOnEr,abâdOna,abâdOn6rô,abâdOn6rE,abâdOn6re,...,abâdOne,abâdOne,abâdOne,abâdOne,NaN,NaN,NaN,abâdOn,abâdOn,abâdOn
2,abasourdir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,abazurdi,abazurdi,abazurdi,NaN,NaN,NaN,NaN,NaN,NaN
3,abattre,NaN,abati,NaN,NaN,abatir,abati,abatrô,abatrE,abatre,...,abaty,abaty,abaty,abaty,NaN,abat,NaN,abat,NaN,abat
4,abdiquer,NaN,abdikE,NaN,NaN,NaN,abdika,NaN,NaN,NaN,...,NaN,abdike,NaN,abdike,NaN,NaN,NaN,NaN,NaN,NaN
5,aberrer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,abhorrer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,abOre,abOre,abOre,NaN,NaN,NaN,NaN,NaN,NaN
7,abjurer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,abZyre,NaN,NaN,NaN,NaN,NaN,NaN
8,abolir,NaN,NaN,NaN,NaN,NaN,abOli,NaN,NaN,NaN,...,abOli,abOli,abOli,abOli,NaN,NaN,NaN,NaN,NaN,NaN
9,abonder,NaN,abôdE,NaN,NaN,NaN,abôda,NaN,NaN,NaN,...,NaN,NaN,NaN,abôde,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
dfVerbes3=verbesLex3.pivot_table(values='freq', 
                                 index=['lexeme'], 
                                 columns=['case'], 
#                                 aggfunc=lambda x: ",".join(x)
                                ).reset_index().reindex()
dfVerbes3

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abaisser,NaN,1.700001e+07,NaN,NaN,4.100001e+07,2.640001e+08,NaN,1.700001e+07,NaN,...,1.000017e+06,4.400006e+07,3.700004e+07,2.090003e+08,NaN,NaN,NaN,NaN,NaN,NaN
1,abandonner,2.700001e+07,2.430001e+08,NaN,1.400000e+07,1.580001e+08,9.510009e+08,5.300003e+07,2.800001e+08,1.400005e+07,...,3.970001e+08,1.700001e+09,8.480003e+08,4.648003e+09,NaN,NaN,NaN,1.094776e+07,9.275606e+06,9.041923e+07
2,abasourdir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.600001e+07,2.700001e+07,2.440000e+08,NaN,NaN,NaN,NaN,NaN,NaN
3,abattre,NaN,2.700004e+07,NaN,NaN,9.800004e+07,5.410004e+08,1.100001e+07,3.900005e+07,2.600002e+07,...,7.100005e+07,4.620002e+08,4.850001e+08,1.891001e+09,NaN,5.738388e+05,NaN,1.100001e+07,NaN,1.184262e+08
4,abdiquer,NaN,7.000002e+06,NaN,NaN,NaN,2.000002e+07,NaN,NaN,NaN,...,NaN,7.000008e+06,NaN,1.060000e+08,NaN,NaN,NaN,NaN,NaN,NaN
5,aberrer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,abhorrer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.000004e+06,7.000003e+06,2.800002e+07,NaN,NaN,NaN,NaN,NaN,NaN
7,abjurer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.100003e+07,NaN,NaN,NaN,NaN,NaN,NaN
8,abolir,NaN,NaN,NaN,NaN,NaN,3.775500e+07,NaN,NaN,NaN,...,4.500001e+07,6.000003e+07,1.257257e+07,2.520001e+08,NaN,NaN,NaN,NaN,NaN,NaN
9,abonder,NaN,7.000004e+06,NaN,NaN,NaN,2.700003e+07,NaN,NaN,NaN,...,NaN,NaN,NaN,2.100007e+07,NaN,NaN,NaN,NaN,NaN,NaN


# Verbes avec des distributions similaires à un défectif de référence

In [20]:
def getSimilairesDefectif(defectif,gDF):
    result={"et":{},"par":{}}
    parDefectif=gDF[gDF["lexeme"]==defectif].squeeze().dropna()
    if len(parDefectif)>0:
        casesDefectif=parDefectif.index.tolist()
        result["par"]["casesP"]=casesDefectif

        etDefectif=[e for e in etCells if set(e)&set(casesDefectif)]
        casesDefectifET=["lexeme"]
        for et in etDefectif:
            casesDefectifET.extend(et)
        result["et"]["casesP"]=casesDefectifET

        casesDefectifVides=[c for c in casesVerbes if c not in casesDefectif]
        result["par"]["casesV"]=casesDefectifVides
        casesDefectifETVides=[c for c in casesVerbes if c not in casesDefectifET]
        result["et"]["casesV"]=casesDefectifETVides

        verbesDefectifMin=gDF[casesDefectif].dropna().lexeme.tolist()
        verbesDefectifMax=gDF[gDF[casesDefectifVides].isnull().all(axis=1)].lexeme.tolist()
        result["par"]["vMin"]=set(verbesDefectifMin)
        result["par"]["vMax"]=set(verbesDefectifMax)

        verbesDefectifETMin=gDF[casesDefectifET].dropna().lexeme.tolist()
        verbesDefectifETMax=gDF[gDF[casesDefectifETVides].isnull().all(axis=1)].lexeme.tolist()
        result["et"]["vMin"]=set(verbesDefectifETMin)
        result["et"]["vMax"]=set(verbesDefectifETMax)
    else:
        print lexeme,"paradigme vide"
        print
        result=None
    return result

In [38]:
boyeDefectifs=u"abstraire accroire adirer advenir apparoir assavoir".split(" ")
boyeDefectifs+=u"avenir becter béer bienvenir braire bruire chaloir".split(" ")
boyeDefectifs+=u"choir clore comparoir contrefoutre courre déchoir".split(" ")
boyeDefectifs+=u"déclore dépourvoir discontinuer dissoudre".split(" ")
boyeDefectifs+=u"distraire douer échoir éclore".split(" ")
boyeDefectifs+=u"enclore endêver ensuivre ester extraire faillir".split(" ")
boyeDefectifs+=u"férir forclore forfaire foutre frire gésir".split(" ")
boyeDefectifs+=u"huir impartir issir malfaire méfaire messeoir".split(" ")
boyeDefectifs+=u"moufeter moufter mourir oindre ouïr paître parfaire".split(" ")
boyeDefectifs+=u"poindre quérir raire rassir ravoir reclure revaloir".split(" ")
boyeDefectifs+=u"saillir seoir sourdre soustraire stupéfaire".split(" ")
boyeDefectifs+=u"surfaire titre traire urger".split(" ")
print '[u"'+'", u"'.join(boyeDefectifs)+'"]'

[u"abstraire", u"accroire", u"adirer", u"advenir", u"apparoir", u"assavoir", u"avenir", u"becter", u"béer", u"bienvenir", u"braire", u"bruire", u"chaloir", u"choir", u"clore", u"comparoir", u"contrefoutre", u"courre", u"déchoir", u"déclore", u"dépourvoir", u"discontinuer", u"dissoudre", u"distraire", u"douer", u"échoir", u"éclore", u"enclore", u"endêver", u"ensuivre", u"ester", u"extraire", u"faillir", u"férir", u"forclore", u"forfaire", u"foutre", u"frire", u"gésir", u"huir", u"impartir", u"issir", u"malfaire", u"méfaire", u"messeoir", u"moufeter", u"moufter", u"mourir", u"oindre", u"ouïr", u"paître", u"parfaire", u"poindre", u"quérir", u"raire", u"rassir", u"ravoir", u"reclure", u"revaloir", u"saillir", u"seoir", u"sourdre", u"soustraire", u"stupéfaire", u"surfaire", u"titre", u"traire", u"urger"]


In [21]:
defectifSimilaires={}
for lexeme in boyeDefectifs:
    dictDefectif=getSimilairesDefectif(lexeme,dfVerbes3)
    if dictDefectif:
        defectifSimilaires[lexeme]=list(dictDefectif["et"]["vMax"]&dictDefectif["par"]["vMin"])
        print lexeme, ", ".join([c for c in dictDefectif["par"]["casesP"] if c!="lexeme"])
        print ", ".join(list(dictDefectif["et"]["vMax"]&dictDefectif["par"]["vMin"]))
        print

abstraire inf, pi1S, pi3S, ppFP, ppFS, ppMP, ppMS
baguer, délabrer, abstraire, parfaire

accroire inf
équeuter, remprunter, contremander, ressemer, correctionnaliser, shampooiner, vulgariser, resalir, criminaliser, cancériser, entrebattre, lapiner, réabonner, recarreler, décriminaliser, coqueter, signaliser, revoter, rober, reloquer, désensabler, rapointir, bluter, maroufler, enkyster, désincruster, réséquer, rentamer, empanner, désinsectiser, engraver, jodler, désengourdir, enclouer, entr'aimer, cureter, reclure, quarter, tictaquer, maquereauter, assavoir, dansotter, accroire, quérir, paperasser, reposséder, dénazifier, retuber, débobiner, dépiler, délustrer, alcaliniser, escher, pleuvioter, ravoir, pasteuriser, égrainer, recorder, tartir, scolariser, partouser, dépolluer, comparoir, avitailler, encliqueter, bayer, embraquer, courre, inférioriser, rassir, démurger, recongeler, zester, ambler, décaisser, varapper, rebander, crémer, décuver, occire, boyauter, désaccoupler, exemplifier, 

discontinuer ii3P, inf
splitter, parloter, discontinuer, sonnailler, encorner, grognasser, volter, retisser, remouiller

dissoudre fi1S, fi3P, fi3S, ii3P, ii3S, inf, pI2S, pP, pc3S, pi1S, pi2S, pi3P, pi3S, ppMP, ppMS, ps2S, ps3S
dissoudre

dissoudre fi1S, fi3P, fi3S, ii3P, ii3S, inf, pI2S, pP, pc3S, pi1S, pi2S, pi3P, pi3S, ppMP, ppMS, ps2S, ps3S
dissoudre

distraire fi3P, fi3S, ii1S, ii3P, ii3S, inf, pI1P, pI2P, pI2S, pP, pc3P, pc3S, pi1P, pi1S, pi2P, pi2S, pi3P, pi3S, ppFP, ppFS, ppMP, ppMS, ps1S, ps2S, ps3P, ps3S
distraire

douer inf, ppFP, ppFS, ppMP, ppMS
douer, enclore, informatiser, graduer, prédéterminer, proportionner, éculer, désaffecter, vernisser, plastifier, stéréotyper, diplômer, endolorir, désabuser, promouvoir, insonoriser, oranger, oindre, téléguider, gratiner, aromatiser, nantir, corseter, gominer, motoriser

échoir paradigme vide

éclore paradigme vide

enclore inf, ppFP, ppFS, ppMP, ppMS
douer, enclore, informatiser, graduer, prédéterminer, proportionner, éculer, d

soustraire ii3P, ii3S, inf, pI2P, pI2S, pP, pc3S, pi1S, pi3S, ppFP, ppFS, ppMP, ppMS
soustraire, brutaliser

stupéfaire paradigme vide

surfaire ii3S, inf, ppFS, ppMP
surfaire, polycopier, démâter, gauchir, endimancher, transmuer, salarier, chaperonner, capitonner, désappointer, givrer, défraîchir, crotter, galber, piqueter, enrubanner, violacer, raciner, déboussoler, ankyloser, boucaner, désemparer

titre paradigme vide

traire fi3S, ii3S, inf, pP, pc3S, pi1S, pi2S, pi3S, ppMP, ppMS, ps1S
traire

urger ii3S, inf, pi3P, pi3S, ps3P, ps3S
urger



In [25]:
defectifSimilaires["clore"]

['clore']

# Tableaux de conjugaison des voisins de défectifs

In [42]:
result=""
for d in defectifSimilaires:
    if 25>len(defectifSimilaires[d])>1:
        print d#,defectifSimilaires[d]
        result+=u"\\subsection{%s}\n"%d
        for v in defectifSimilaires[d]:
            verbe=v.decode("utf8")
            verbeTab=makeTabularParadigmeDF(verbe,dfParadigmes3,dictEtColours)
            result+=verbeTab
pyperclip.copy(result)


surfaire
abstraire
becter
parfaire
soustraire
discontinuer


In [29]:
result=""
for v in u"retravailler cautionner".split(" "):
    result+=makeTabularParadigmeDF(v,dfParadigmes3,dictEtColours)
pyperclip.copy(result)

In [66]:
pyperclip.copy(makeTabularParadigmeDF(u"distraire",dfGold3,dictEtColours))

In [67]:
pyperclip.copy(makeTabularParadigmeDF(u"distraire",dfParadigmes3,dictEtColours))

In [65]:
print dfVerbes3[dfVerbes3["lexeme"]=="distraire"].T.dropna().sort_values(1306,ascending=False).to_latex()

\begin{tabular}{ll}
\toprule
{} &         1306 \\
case   &              \\
\midrule
lexeme &    distraire \\
inf    &    2.094e+09 \\
pi3S   &  7.14722e+08 \\
ppMS   &  4.06279e+08 \\
ppFS   &      3.7e+08 \\
ii3S   &     1.57e+08 \\
ppMP   &     1.03e+08 \\
fi3S   &      7.5e+07 \\
pi3P   &  6.28479e+07 \\
pi1S   &  6.25845e+07 \\
ii3P   &  5.40001e+07 \\
pP     &  4.60001e+07 \\
pc3S   &      4.6e+07 \\
ii1S   &  3.10001e+07 \\
pi2S   &  3.05712e+07 \\
pi2P   &  3.00774e+07 \\
ppFP   &      2.5e+07 \\
ps3S   &  2.18474e+07 \\
pi1P   &  1.63298e+07 \\
fi3P   &  8.00001e+06 \\
ps2S   &        8e+06 \\
pc3P   &  7.00001e+06 \\
pI2P   &  3.92275e+06 \\
pI2S   &  2.84476e+06 \\
pI1P   &  1.67027e+06 \\
ps3P   &  1.15228e+06 \\
ps1S   &       152651 \\
\bottomrule
\end{tabular}



In [79]:
dfVerbes3

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abaisser,NaN,1.700001e+07,NaN,NaN,4.100001e+07,2.640001e+08,NaN,1.700001e+07,NaN,...,1.000017e+06,4.400006e+07,3.700004e+07,2.090003e+08,NaN,NaN,NaN,NaN,NaN,NaN
1,abandonner,2.700001e+07,2.430001e+08,NaN,1.400000e+07,1.580001e+08,9.510009e+08,5.300003e+07,2.800001e+08,1.400005e+07,...,3.970001e+08,1.700001e+09,8.480003e+08,4.648003e+09,NaN,NaN,NaN,1.094776e+07,9.275606e+06,9.041923e+07
2,abasourdir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.600001e+07,2.700001e+07,2.440000e+08,NaN,NaN,NaN,NaN,NaN,NaN
3,abattre,NaN,2.700004e+07,NaN,NaN,9.800004e+07,5.410004e+08,1.100001e+07,3.900005e+07,2.600002e+07,...,7.100005e+07,4.620002e+08,4.850001e+08,1.891001e+09,NaN,5.738388e+05,NaN,1.100001e+07,NaN,1.184262e+08
4,abdiquer,NaN,7.000002e+06,NaN,NaN,NaN,2.000002e+07,NaN,NaN,NaN,...,NaN,7.000008e+06,NaN,1.060000e+08,NaN,NaN,NaN,NaN,NaN,NaN
5,aberrer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,abhorrer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.000004e+06,7.000003e+06,2.800002e+07,NaN,NaN,NaN,NaN,NaN,NaN
7,abjurer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.100003e+07,NaN,NaN,NaN,NaN,NaN,NaN
8,abolir,NaN,NaN,NaN,NaN,NaN,3.775500e+07,NaN,NaN,NaN,...,4.500001e+07,6.000003e+07,1.257257e+07,2.520001e+08,NaN,NaN,NaN,NaN,NaN,NaN
9,abonder,NaN,7.000004e+06,NaN,NaN,NaN,2.700003e+07,NaN,NaN,NaN,...,NaN,NaN,NaN,2.100007e+07,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
dfVerbes3[(dfVerbes3[etCells[10]].isnull().all(axis=1))&
          (~dfVerbes3.lexeme.isin(boyeDefectifs))
#          &
#          (dfVerbes3.lexeme.str.endswith("er"))
         ].lexeme

2         abasourdir
5            aberrer
6           abhorrer
7            abjurer
12          aboucher
13           abouler
14           abouter
17           abraser
19         abricoter
21           abroger
26          absoudre
30         abâtardir
38        accidenter
41         accointer
52         accourcir
54         accoutrer
67        achalander
73          aciduler
80        actualiser
81            acérer
83       additionner
86           adjuger
92          adoniser
100           aduler
101        adultérer
103        affabuler
109          affamer
111     affectionner
114          affiler
122       affouiller
            ...     
5283      émulsifier
5284     émulsionner
5285         émécher
5286       énamourer
5289        énucléer
5293         épandre
5301         épeurer
5302          épicer
5304          épiler
5309        épointer
5311       épouiller
5315    époustoufler
5321          épurer
5322        équarrir
5323        équeuter
5327       éradiquer
5330        é

In [ ]:
dfVerbes3[dfVerbes3.lexeme.isin(verbesDefectifETMax)][casesDefectifET]